In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import train
import model
import evaluate
import beam_search
import data_util.data
import data_util.batcher
import data_util.config


importlib.reload(model)
importlib.reload(train)
importlib.reload(evaluate)
importlib.reload(beam_search)
importlib.reload(data_util.config)
importlib.reload(data_util.data)
importlib.reload(data_util.batcher)

from model import *
from train import *
from beam_search import *
from evaluate import *
from data_util.data import *
from data_util.batcher import *


In [3]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data.tsv'
dat = pd.read_csv(data_path, sep='\t')

# test/train split
x_train, x_test, y_train, y_test = train_test_split(
    dat.TaskSentence.values, 
    dat.Summary.values,
    test_size=0.2,
    random_state=0,
    shuffle=True
)

# fit tokenizer
tokenizer = Tokenizer(lower=True, filters='!"#$%&()*+,./:;<=>?[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(list(x_train)+list(y_train))

# lowercase the data
x_train = tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(x_train))
y_train = tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(y_train))

examples = list(zip(x_train, y_train))
# examples

In [4]:
random.seed(123)
T.manual_seed(123)
if T.cuda.is_available():
    T.cuda.manual_seed_all(123)
    
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [5]:
# Standards
# Make Vocab
vocab = data.Vocab(
    words = tokenizer.word_index.keys(), 
    max_size=len(tokenizer.word_index))

# Make Batcher
batcher = Batcher(
    examples=examples,
    vocab=vocab, 
    mode='train', 
    batch_size=32, 
    single_pass=False)

max_size of vocab was specified as 2829; we now have 2829 words. Stopping reading.
Finished constructing vocabulary of 2829 total words. Last word added: foundation


In [6]:
train_mle = "yes"
train_rl = "no"
mle_weight = 1.0
load_model = None
new_lr = None
rl_weight = 1 - mle_weight

opt = Namespace(train_mle = train_mle, 
                train_rl = train_rl, 
                mle_weight = mle_weight, 
                load_model = load_model,
                new_lr = new_lr, 
                rl_weight = rl_weight)


train_processor = Train(vocab, batcher, opt)

In [7]:
# train_processor.setup_train(model=Model)
# train_processor.train_one_batch(train_processor.batcher.next_batch(), 1)

In [83]:
task = "validate"
load_model = os.path.join(config.log_root, "data/saved_models/0000500.tar")

opt = Namespace(task = task, load_model = load_model)

# new batcher for evaluation
batcher = Batcher(
    examples=examples[:20],
    vocab=vocab, 
    mode='train', 
    batch_size=10, 
    single_pass=True)

eval_processor = Evaluate(vocab, batcher, opt) 

example_generator completed reading all examples. No more data.


Exception in thread Thread-68:
Traceback (most recent call last):
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 273, in text_generator
    e = next(example_generator) # e is a tf.Example
StopIteration

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 207, in fill_example_queue
    (article, abstract) = next(input_gen) # read the next example from file. article and abstract are both strings.
RuntimeError: generator raised StopIteration



INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [84]:
decoded_sents, ref_sents = eval_processor.evaluate_batch(model=Model)

Summarizing Batch...
tensor([[  2, 407],
        [  2,  46],
        [  2,  96],
        [  2, 140]])
tensor([[  2,  16],
        [  2, 105],
        [  2,  19],
        [  2,  14]])
tensor([[  2, 320],
        [  2,  46],
        [  2,  19],
        [  2,   5]])
tensor([[  2,  55],
        [  2, 248],
        [  2,  24],
        [  2,  19]])
tensor([[ 2, 28],
        [ 2, 19],
        [ 2, 46],
        [ 2, 24]])
tensor([[ 2, 95],
        [ 2, 19],
        [ 2, 46],
        [ 2,  9]])
tensor([[ 2, 39],
        [ 2, 19],
        [ 2, 46],
        [ 2, 24]])
tensor([[  2, 140],
        [  2,  46],
        [  2,  19],
        [  2,  67]])
tensor([[  2,  59],
        [  2, 248],
        [  2,  96],
        [  2, 138]])
tensor([[  2, 575],
        [  2,  46],
        [  2,  96],
        [  2, 140]])
tensor([[  2, 407, 321],
        [  2,  46, 321],
        [  2,  96, 321],
        [  2, 140, 321]])
tensor([[   2,   16,    9],
        [   2,   16,   31],
        [   2,   16,   10],
        

tensor([[  2,  46,   9, 481, 481, 481,   4,   4,   3,   3],
        [  2,  46,   9, 481, 481,  48,   4,   4,   3,   3],
        [  2,  46,   9, 481, 481,  48,   4,   4,   4,   3],
        [  2,  46,   9,  11, 481,  48,   4,   4,   4,   3]])
INFO:tensorflow:Finished reading dataset in single_pass mode.
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [85]:
decoded_sents

["insert appropriate seller's payment",
 'call sender',
 'save power point',
 'look this',
 'email sender',
 'change it',
 'give sender to sender',
 'complete and and sender',
 'get with ondetails',
 'communicate with mark holsworth',
 'reply to sender about sender',
 'inform sender of to load load gtcs',
 'review suggested mark mark',
 'give number of copies',
 'make pg e topock curves are flat socal',
 'contact sender further further further to to',
 'remind sender',
 'look at book review',
 'return toaster',
 'forward this to jon']

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [86]:
ref_sents

["insert appropriate seller's payment",
 'call sender',
 'save power point',
 'examine this',
 'send findings to sender',
 'change calendar',
 'respond to sender with comments about it',
 'complete document and return to sender',
 'give patti ondetails',
 'communicate with mark holsworth',
 'reply to sender about ocr program',
 'inform sender of needs to load the gtcs',
 'review suggested mark up',
 'give number of copies to sender',
 'verify pg e topock curves are flat to socal',
 'contact sender with further comments and to coordinate execution',
 'remind sender when in banff louise',
 'look at book review',
 'return toaster to sender',
 'forward it to jon']

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


## Now Repeating with the BERT model

In [115]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

In [8]:
bert_tokenizer = BertTokenizer.from_pretrained(bert_name)

In [ ]:
# update 
for t in [START_DECODING, STOP_DECODING]:
    bert_tokenizer.vocab[t] = len(bert_tokenizer.vocab)

In [ ]:
bert_model = BertModel.from_pretrained(bert_config)
bert_model.resize_token_embeddings(len(bert_tokenizer)) 

In [118]:
class BertVocab(Vocab):
  def __init__(self, bert_name):
    self.tokenizer = bert_tokenizer
    self._word_to_id = self.tokenizer.vocab
    self._id_to_word = self.tokenizer.ids_to_tokens
    self._count = self.tokenizer.vocab_size
    


INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [162]:
bt.vocab[bt.cls_token]

KeyError: '[START]'

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket qu

In [160]:
bt.add_special_tokens({'cls_token': '[START]'})

1

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [161]:
bt.cls_token

'[START]'

In [87]:
# Make Vocab
bert_vocab = BertVocab('bert-base-uncased')

# Make Batcher
bert_batcher = Batcher(
    examples=examples,
    vocab=bert_vocab, 
    mode='train', 
    batch_size=32, 
    single_pass=False)

INFO:tensorflow:Bucket queue size: 23, Input queue size: 204


In [92]:
train_mle = "yes"
train_rl = "no"
mle_weight = 1.0
load_model = None
new_lr = None
rl_weight = 1 - mle_weight

opt = Namespace(train_mle = train_mle, 
                train_rl = train_rl, 
                mle_weight = mle_weight, 
                load_model = load_model,
                new_lr = new_lr, 
                rl_weight = rl_weight)


bert_train_processor = Train(bert_vocab, bert_batcher, opt)

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [93]:
# bert_train_processor.setup_train(model=BertSummarizer)
# bert_train_processor.train_one_batch(bert_train_processor.batcher.next_batch(), 1)

In [95]:
config.save_model_path = "data/bert_saved_models"

bert_train_processor.trainIters(n_iters=2, model=BertSummarizer, report_every=1, save_every = 2)

0
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
iter: 1 mle_loss: 6.451 reward: 0.0000
1
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
iter: 2 mle_loss: 6.358 reward: 0.0000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
model saved at: 
 data/bert_saved_models/0000002.tar
2
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
iter: 3 mle_loss: 6.076 reward: 0.0000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Inp

Yay!

In [96]:
task = "validate"
load_model = os.path.join(config.log_root, "data/bert_saved_models/0000002.tar")

opt = Namespace(task = task, load_model = load_model)

# new batcher for evaluation
bert_decode_batcher = Batcher(
    examples=examples[:20],
    vocab=bert_vocab, 
    mode='train', 
    batch_size=10, 
    single_pass=True)

eval_processor = Evaluate(bert_vocab, bert_decode_batcher, opt) 

example_generator completed reading all examples. No more data.


Exception in thread Thread-77:
Traceback (most recent call last):
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 273, in text_generator
    e = next(example_generator) # e is a tf.Example
StopIteration

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 207, in fill_example_queue
    (article, abstract) = next(input_gen) # read the next example from file. article and abstract are both strings.
RuntimeError: generator raised StopIteration



In [97]:
decoded_sents, ref_sents = eval_processor.evaluate_batch(model=BertSummarizer)

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
Summarizing Batch...
tensor([[  100,  3531],
        [  100, 19274],
        [  100,  6413],
        [  100, 30522]])
tensor([[  100,  2021],
        [  100, 30522],
        [  100,  2145],
        [  100,  2182]])
tensor([[  100, 30522],
        [  100,  3531],
        [  100,  3828],
        [  100,  2023]])
tensor([[ 100, 5736],
        [ 100, 2202],
        [ 100, 1037],
        [ 100, 4248]])
tensor([[  100,  3531],
        [  100, 10373],
        [  100,  2033],
        [  100,  2054]])
tensor([[ 100, 2064],
        [ 100, 2017],
        [ 100, 2689],
        [ 100, 2009]])
tensor([[ 100, 3531],
        [ 100, 2507],
        [ 100, 2033],
        [ 100, 2115]])
tensor([[ 100, 2065],
        [ 100, 3100],
        [ 100, 3531]

tensor([[  100,  2021,  1037,  1037,  1037,  1037,  1037,  1037],
        [  100, 30522,  1037,  1037,  1037,  1037,  1037,  1037],
        [  100,  2145,  1037,  1037,  1037,  1037,  1037,  1037],
        [  100,  2021,  1037,  1037,  1037,  1037,  1037, 30523]])
tensor([[  100, 30522,  2031,  2031,  2031,  2031,  2031,  2031],
        [  100, 30522,  2017,  2031,  2031,  2031,  2031,  2031],
        [  100, 30522, 14176,  2031,  2031,  2031,  2031,  2031],
        [  100, 30522,  2391,  2031,  2031,  2031,  2031,  2031]])
tensor([[ 100, 5736, 2017, 2017, 2017, 2017, 2017, 2017],
        [ 100, 5736, 2052, 2017, 2017, 2017, 2017, 2017],
        [ 100, 5736, 2023, 2017, 2017, 2017, 2017, 2017],
        [ 100, 5736, 2017, 2052, 2017, 2017, 2017, 2017]])
tensor([[ 100, 3531, 2041, 2041, 2041, 2041, 2041, 2041],
        [ 100, 3531, 2424, 2041, 2041, 2041, 2041, 2041],
        [ 100, 3531, 2017, 2041, 2041, 2041, 2041, 2041],
        [ 100, 3531, 2041, 2424, 2041, 2041, 2041, 2041]])
tens

Summarizing Batch...
tensor([[ 100, 2065],
        [ 100, 2017],
        [ 100, 2024],
        [ 100, 2306]])
tensor([[  100,  2320],
        [  100,  6294],
        [  100, 30522],
        [  100,  2013]])
tensor([[  100, 30522],
        [  100,  3531],
        [  100,  2202],
        [  100,  1037]])
tensor([[ 100, 3531],
        [ 100, 2507],
        [ 100, 2033],
        [ 100, 1996]])
tensor([[ 100, 3531],
        [ 100, 2191],
        [ 100, 3056],
        [ 100, 2008]])
tensor([[ 100, 3531],
        [ 100, 3967],
        [ 100, 2004],
        [ 100, 2149]])
tensor([[  100, 10825],
        [  100,  2033],
        [  100,  2043],
        [  100,  2017]])
tensor([[ 100, 3531],
        [ 100, 2202],
        [ 100, 1037],
        [ 100, 2298]])
tensor([[ 100, 2052],
        [ 100, 2017],
        [ 100, 3531],
        [ 100, 2709]])
tensor([[ 100, 2071],
        [ 100, 2017],
        [ 100, 2830],
        [ 100, 2023]])
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


tensor([[ 100, 2065, 2017, 2033, 2033, 2033, 1998, 1998],
        [ 100, 2065, 2017, 2033, 2033, 1998, 1998, 1998],
        [ 100, 2065, 2033, 2033, 2033, 2033, 1998, 1998],
        [ 100, 2065, 2024, 2033, 2033, 2033, 1998, 1998]])
tensor([[ 100, 2320, 2000, 2000, 2000, 1996, 1996, 1996],
        [ 100, 2320, 2000, 2000, 1996, 1996, 1996, 1996],
        [ 100, 2320, 2000, 2000, 2000, 2000, 1996, 1996],
        [ 100, 2320, 2000, 2000, 1996, 2000, 1996, 1996]])
tensor([[  100, 30522,  2113, 11968, 11968, 11968, 11968, 11968],
        [  100, 30522,  2033, 11968, 11968, 11968, 11968, 11968],
        [  100, 30522,  2065, 11968, 11968, 11968, 11968, 11968],
        [  100, 30522,  2292, 11968, 11968, 11968, 11968, 11968]])
tensor([[  100,  3531,  1045, 19575, 19575, 19575, 19575, 19575],
        [  100,  3531,  1998, 19575, 19575, 19575, 19575, 19575],
        [  100,  3531,  2064, 19575, 19575, 19575, 19575, 19575],
        [  100,  3531,  4809, 19575, 19575, 19575, 19575, 19575]])
tens

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [105]:
data.START_DECODING

'[START]'

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [104]:
eval_processor.vocab.word2id(data.START_DECODING)

100

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [103]:
eval_processor.vocab.id2word(10825)

'remind'

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket qu

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [72]:
decoded_sents

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket qu

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket qu

In [60]:
ref_sents

["insert appropriate seller's payment",
 'call sender',
 'save power point',
 'examine this',
 'send findings to sender',
 'change calendar',
 'respond to sender with comments about it',
 'complete document and return to sender',
 'give patti ondetails',
 'communicate with mark holsworth',
 'reply to sender about ocr program',
 'inform sender of needs to load the gtcs',
 'review suggested mark up',
 'give number of copies to sender',
 'verify pg e topock curves are flat to socal',
 'contact sender with further comments and to coordinate execution',
 'remind sender when in banff louise',
 'look at book review',
 'return toaster to sender',
 'forward it to jon']

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket qu